In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.4` // Or use any other 2.x version here
// import $ivy.`sh.almond::almond-spark:0.10.9` 

import $ivy.$                                   // Or use any other 2.x version here
// import $ivy.`sh.almond::almond-spark:0.10.9` 

In [2]:
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)


import org.apache.log4j.{Level, Logger}


In [7]:
import org.apache.spark.sql._
import spark.implicits._

import org.apache.spark.sql._

import spark.implicits._

In [8]:
val spark = {
  SparkSession.builder()
    .master("local[*]")
    .config("spark.testing.memory", 471859200)
    .config("spark.executor.instances", "4")
    .config("spark.executor.memory", "2g")
    .getOrCreate()
}

spark: SparkSession = org.apache.spark.sql.SparkSession@79f04275

In [9]:
val df = spark.read.format("parquet").load("data/train")

df: DataFrame = [instanceId_userId: int, instanceId_objectType: string ... 167 more fields]

In [10]:
//  посмотрим на список колонок
for (c <- df.columns) {
    println(c)
}

instanceId_userId
instanceId_objectType
instanceId_objectId
audit_pos
audit_clientType
audit_timestamp
audit_timePassed
audit_experiment
audit_resourceType
metadata_ownerId
metadata_ownerType
metadata_createdAt
metadata_authorId
metadata_applicationId
metadata_numCompanions
metadata_numPhotos
metadata_numPolls
metadata_numSymbols
metadata_numTokens
metadata_numVideos
metadata_platform
metadata_totalVideoLength
metadata_options
relationsMask
userOwnerCounters_USER_FEED_REMOVE
userOwnerCounters_USER_PROFILE_VIEW
userOwnerCounters_VOTE_POLL
userOwnerCounters_USER_SEND_MESSAGE
userOwnerCounters_USER_DELETE_MESSAGE
userOwnerCounters_USER_INTERNAL_LIKE
userOwnerCounters_USER_INTERNAL_UNLIKE
userOwnerCounters_USER_STATUS_COMMENT_CREATE
userOwnerCounters_PHOTO_COMMENT_CREATE
userOwnerCounters_MOVIE_COMMENT_CREATE
userOwnerCounters_USER_PHOTO_ALBUM_COMMENT_CREATE
userOwnerCounters_COMMENT_INTERNAL_LIKE
userOwnerCounters_USER_FORUM_MESSAGE_CREATE
userOwnerCounters_PHOTO_MARK_CREATE
userOwnerCoun

In [31]:
df.createOrReplaceTempView("trainDF")
val df_exp = spark.sql("""
    select t.*, explode(feedback) as fb_exp from trainDF t 
""")
df_exp.createOrReplaceTempView("trainDF_exp")

df_exp: DataFrame = [instanceId_userId: int, instanceId_objectType: string ... 168 more fields]

# Задание 1 - Постройте топ популярных групп на портале

In [32]:
spark.sql("""
    select metadata_ownerId, count(*) as like_cnt from trainDF_exp
    where fb_exp = 'Liked'
    group by metadata_ownerId
    order by like_cnt desc
""").show(10)

+----------------+--------+
|metadata_ownerId|like_cnt|
+----------------+--------+
|           37463|   16726|
|           11222|   12301|
|           18942|   10160|
|           14285|   10024|
|           62833|    9428|
|           26843|    8779|
|           19481|    8717|
|           79619|    8519|
|           11058|    8384|
|           26735|    7770|
+----------------+--------+
only showing top 10 rows



In [38]:
spark.sql("""
    select distinct fb_exp from trainDF_exp
""").show()

+---------+
|   fb_exp|
+---------+
|Commented|
|   Viewed|
| ReShared|
|  Clicked|
|Complaint|
|  Ignored|
| Disliked|
|    Liked|
|  Unliked|
+---------+



In [46]:
val activityDF = spark.sql("""
    select created_hour, count(*) as cnt from (
        select 
            from_unixtime(metadata_createdAt, 'HH') as created_hour
            , fb_exp
        from trainDF_exp 
        where fb_exp <> 'Ignored'
    ) 
    group by created_hour
    order by created_hour
""")

val popularityDF = spark.sql("""
    select created_hour, count(*) as cnt from (
        select 
            from_unixtime(metadata_createdAt, 'HH') as created_hour
            , fb_exp
        from trainDF_exp 
        where fb_exp = 'Liked'
    ) 
    group by created_hour
    order by created_hour
""")



activityDF: DataFrame = [created_hour: string, cnt: bigint]
popularityDF: DataFrame = [created_hour: string, cnt: bigint]

In [47]:
activityDF.show(24)

+------------+------+
|created_hour|   cnt|
+------------+------+
|          00|312314|
|          01|321707|
|          02|319148|
|          03|315376|
|          04|325096|
|          05|320993|
|          06|313794|
|          07|340855|
|          08|312593|
|          09|323576|
|          10|323072|
|          11|317125|
|          12|316304|
|          13|322514|
|          14|303189|
|          15|328408|
|          16|316001|
|          17|325420|
|          18|326375|
|          19|328067|
|          20|335332|
|          21|323341|
|          22|307189|
|          23|328270|
+------------+------+



In [48]:
popularityDF.show(24)

+------------+------+
|created_hour|   cnt|
+------------+------+
|          00|131087|
|          01|134482|
|          02|131933|
|          03|132778|
|          04|136470|
|          05|135081|
|          06|131198|
|          07|142981|
|          08|131175|
|          09|136853|
|          10|135167|
|          11|132435|
|          12|130546|
|          13|135089|
|          14|127380|
|          15|137408|
|          16|131774|
|          17|134038|
|          18|134195|
|          19|134972|
|          20|139733|
|          21|135906|
|          22|128918|
|          23|138830|
+------------+------+

